# Customer Churn Prediction
## **Question?** Will they churn once their current set of policies expire?
### **Churned**: no policy for (>=6 months)
### **Non-Churned**: has policy for (2 years+) and not without policy for (>=6 months) 

## **Features**:
1. First policy year (static)
2. Sum of premiums in USD (dynamic)
3. Sum of claims in USD (dynamic)
4. Number of policies (dynamic)
5. Number of claims (dynamic)
6. State (mode; static)
7. County (mode; static)
8. Sum of deductibles (dynamic)
9. Average equipment year (dynamic)
10. Average location premium (dynamic)

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Generic Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [28]:
claims_data = '../data/claims.csv'
premiums_data = '../data/premiums.csv'

In [29]:
claims_df = pd.read_csv(claims_data)
premiums_df = pd.read_csv(premiums_data)

In [30]:
claims_df = claims_df[~claims_df['ClaimCause'].isin(['Claim Denied', 'Claim Withdrawn'])]

In [31]:
claims_df = claims_df.dropna()

claims_df['DateOfLoss'] = pd.to_datetime(claims_df['DateOfLoss'])
claims_df['ClaimReceivedDate'] = pd.to_datetime(claims_df['ClaimReceivedDate'])
claims_df['PolicyEffectiveDate'] = pd.to_datetime(claims_df['PolicyEffectiveDate'])

claims_df = claims_df.astype({
    'PolicyNumber': 'string',
    'ClaimCause': 'string',
    'County': 'string',
    'State': 'string'
})

In [32]:
premiums_df = premiums_df.dropna()

premiums_df['PolicyEffectiveDate'] = pd.to_datetime(premiums_df['PolicyEffectiveDate'])
premiums_df['PolicyExpirationDate'] = pd.to_datetime(premiums_df['PolicyExpirationDate'])

premiums_df = premiums_df.astype({
    'PolicyNumber': 'string',
    'County': 'string',
    'State': 'string'
})

In [33]:
premiums_df = premiums_df.drop_duplicates()

In [34]:
from datetime import datetime

def impute_zero_for_year(
    value : float
) -> float:
    current_year = datetime.now().year
    if 1950.0 <= value <= current_year:
        return value
    else:
        return np.NaN

In [35]:
premiums_df['Equip Year'] = premiums_df['Equip Year'].apply(impute_zero_for_year)

## The below logic is only for getting positive samples

In [ ]:
all_positive_customer_data = pd.DataFrame([], columns = [
    'min_policy_date', 'sum_policy_premiums',
    'sum_location_premiums', 'no_previous_equipment',
    'state', 'sum_of_deductibles',
    'avg_equipment_year', 'total_claim_amount', 'churned'
])

unique_customer_ids = premiums_df['CustomerId'].unique()
positive_sample_customer_ids = []

iteration = 0
for customer_id in unique_customer_ids:
    date_list = []
    for index, row in premiums_df.loc[premiums_df['CustomerId'] == customer_id].iterrows():
        start_date = str(row['PolicyEffectiveDate'].date())
        end_date = str(row['PolicyExpirationDate'].date())
    
        date_range = pd.date_range(start_date, end_date).values.astype('datetime64[D]')
        date_strings = np.datetime_as_string(date_range, unit = 'D').tolist()
        
        date_list += date_strings
    
    unique_active_dates = np.unique(date_list)
    
    customer_date_range = pd.DataFrame({
        'ProbeDate': pd.date_range(
            premiums_df['PolicyEffectiveDate'].min(),
            premiums_df['PolicyExpirationDate'].max(),
            freq = 'D'
        )
    })
    
    customer_date_range['CustomerId'] = customer_id
    customer_date_range['ActivePolicy'] = 0
    customer_date_range.loc[customer_date_range['ProbeDate'].isin(unique_active_dates), 'ActivePolicy'] = 1
    
    customer_date_range = customer_date_range.loc[customer_date_range['ProbeDate'] < '2023-01-01']
    
    customer_date_range['ProbePosition'] = 0
    
    def find_switch(
        df : pd.DataFrame,
        col : str
    ) -> list:
        switch_indices = df[df[col].diff() == -1].index
        for idx in switch_indices:
            if (df.loc[idx : idx + 179, col] == 0).all():
                return idx
        return None
    
    index = find_switch(customer_date_range, 'ActivePolicy')

    if index != None:
        customer_date_range.loc[
            [index - 45, index - 90, index - 135, index - 180, index - 225, index - 270, index - 315, index - 360],
            'ProbePosition'
        ] = 1
        positive_sample_customer_ids.append(customer_id)
    else:
        continue
    
    customer_probe_dates = customer_date_range \
                        .loc[customer_date_range['ProbePosition'] == 1][['CustomerId', 'ProbeDate']] \
                        .reset_index(drop = True)
    
    def first(x):
        return x.iloc[0]
    
    def row_count(x):
        return len(x)
    
    all_probe_data = []
    for index, row in customer_probe_dates.iterrows():
        filtered_premium_df = premiums_df.loc[(premiums_df['CustomerId'] == customer_id) & (premiums_df['PolicyEffectiveDate'] < row['ProbeDate'])]
        probe_premium_data = filtered_premium_df.agg({
            'PolicyEffectiveDate': 'min',
            'PolicyPremium': 'sum',
            'LocationPremium': 'sum',
            'Loc': 'max',
            'State': first,
            'Deductible': 'sum',
            'Equip Year': 'mean'
        }).values
    
        filtered_claims_df = claims_df.loc[(claims_df['CustomerId'] == customer_id) & (claims_df['PolicyEffectiveDate'] < row['ProbeDate'])]
        probe_claims_data = filtered_claims_df.agg({
            'TotalPaidToDate': 'sum'
        }).values
    
        probe_data = list(probe_premium_data.astype(str)) + list(probe_claims_data.astype(str)) + ['1']
    
        all_probe_data.append(probe_data)
    
    customer_data = pd.DataFrame(all_probe_data, columns = [
        'min_policy_date', 'sum_policy_premiums',
        'sum_location_premiums', 'no_previous_equipment',
        'state', 'sum_of_deductibles',
        'avg_equipment_year', 'total_claim_amount', 'churned'
    ])
    
    customer_data = customer_data.drop_duplicates()

    all_positive_customer_data = pd.concat([all_positive_customer_data, customer_data], ignore_index = True)

    iteration += 1
    if iteration % 100 == 0:
        print('Finished 100 additional iterations.')

In [ ]:
all_positive_customer_data.head(5)

In [ ]:
all_positive_customer_data.to_csv('../data/churn_modeling/churn_prediction_positive_samples.csv', index = False)

In [ ]:
premiums_df.loc[premiums_df['CustomerId'] == 4241].head(2)

## The below logic is only for getting negative samples

In [ ]:
negative_customer_samples = premiums_df.groupby('CustomerId').agg({
    'PolicyEffectiveDate': 'min',
    'PolicyExpirationDate': 'max'
}).reset_index(drop = False).rename(columns = {
    'PolicyEffectiveDate': 'minPolicyEffectiveDate',
    'PolicyExpirationDate': 'maxPolicyExpirationDate'
})

In [ ]:
negative_customer_samples = negative_customer_samples.loc[~negative_customer_samples['CustomerId'].isin(positive_sample_customer_ids)]

In [ ]:
negative_customer_samples['policy_activity_length'] = negative_customer_samples['maxPolicyExpirationDate'] - negative_customer_samples['minPolicyEffectiveDate']

In [ ]:
negative_customer_samples['policy_activity_length'] = negative_customer_samples['policy_activity_length'].dt.days

In [ ]:
negative_customer_samples = negative_customer_samples.loc[negative_customer_samples['policy_activity_length'] > 365 * 2]

In [ ]:
negative_customer_samples = negative_customer_samples.loc[negative_customer_samples['minPolicyEffectiveDate'] < '2022-01-01']

In [ ]:
all_negative_customer_data = pd.DataFrame([], columns = [
    'min_policy_date', 'sum_policy_premiums',
    'sum_location_premiums', 'no_previous_equipment',
    'state', 'sum_of_deductibles',
    'avg_equipment_year', 'total_claim_amount', 'churned'
])

for _, negative_customer_row in negative_customer_samples.iterrows():
    random_dates_for_customer = pd.to_datetime( \
        random.choices(pd.date_range(negative_customer_row['minPolicyEffectiveDate'], '2022-01-01') \
        .tolist(), k = 10)).values.astype(str)
    
    customer_ids = np.zeros(10) + negative_customer_row['CustomerId']
    
    customer_probe_data = np.vstack((customer_ids.astype(str), random_dates_for_customer)).T

    customer_probe_dates = pd.DataFrame(customer_probe_data, columns = ['CustomerId', 'ProbeDate'])

    customer_probe_dates['CustomerId'] = customer_probe_dates['CustomerId'].astype(float).astype(int)
    customer_probe_dates['ProbeDate'] = pd.to_datetime(customer_probe_dates['ProbeDate']).dt.date.astype(str)
    
    def first(x):
        return x.iloc[0]
    
    def row_count(x):
        return len(x)
    
    all_probe_data = []

    try:
        for index, row in customer_probe_dates.iterrows():
            filtered_premium_df = premiums_df.loc[(premiums_df['CustomerId'] == negative_customer_row['CustomerId']) & (premiums_df['PolicyEffectiveDate'] < str(row['ProbeDate']))]
            
            probe_premium_data = filtered_premium_df.agg({
                'PolicyEffectiveDate': 'min',
                'PolicyPremium': 'sum',
                'LocationPremium': 'sum',
                'Loc': 'max',
                'State': first,
                'Deductible': 'sum',
                'Equip Year': 'mean'
            }).values
        
            filtered_claims_df = claims_df.loc[(claims_df['CustomerId'] == negative_customer_row['CustomerId']) & (claims_df['PolicyEffectiveDate'] <  str(row['ProbeDate']))]
            probe_claims_data = filtered_claims_df.agg({
                'TotalPaidToDate': 'sum'
            }).values
        
            probe_data = list(probe_premium_data.astype(str)) + list(probe_claims_data.astype(str)) + ['0']
        
            all_probe_data.append(probe_data)
        
        customer_data = pd.DataFrame(all_probe_data, columns = [
            'min_policy_date', 'sum_policy_premiums',
            'sum_location_premiums', 'no_previous_equipment',
            'state', 'sum_of_deductibles',
            'avg_equipment_year', 'total_claim_amount', 'churned'
        ])
        
        customer_data = customer_data.drop_duplicates()
        
        all_negative_customer_data = pd.concat([all_negative_customer_data, customer_data], ignore_index = True)
        
        iteration += 1
        if iteration % 100 == 0:
            print('Finished 100 additional iterations.')
    except Exception as e:
        print('Hit a snag.')
        pass

In [ ]:
all_negative_customer_data.head(15)

In [ ]:
all_negative_customer_data.to_csv('../data/churn_modeling/churn_prediction_negative_samples.csv', index = False)